In [1]:
import os
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models import alexnet

In [2]:
model = alexnet()
convs = []
# list of a conv layers within alexnn
for i,k in enumerate(model.modules()):
    if isinstance(k, nn.Conv2d):
        convs.append(k)

In [3]:
l1 = convs[0].weight

In [4]:
for j in range(l1.shape[1]): # iterate over channels
    X = torch.reshape(l1[0][j], [1,l1[0][j].shape[1]**2])  # gets first elem
    for i,w in enumerate(l1): # iterates over filters
        if i == 0:
            continue
        else:
            y = torch.reshape(w[0], [1,w[j].shape[1]**2])
            X = torch.cat((X,y))
# X is a matrix with each line being a filter

In [5]:
# Returns the covariance matrix of m
def cov(m, rowvar=False):
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()

In [6]:
def _batch_mahalanobis(L, x):
    r"""
    Computes the squared Mahalanobis distance :math:`\mathbf{x}^\top\mathbf{M}^{-1}\mathbf{x}`
    for a factored :math:`\mathbf{M} = \mathbf{L}\mathbf{L}^\top`.

    Accepts batches for both L and x.
    """
    # TODO: use `torch.potrs` or similar once a backwards pass is implemented.
    flat_L = L.unsqueeze(0).reshape((-1,) + L.shape[-2:])
    L_inv = torch.stack([torch.inverse(Li.t()) for Li in flat_L]).view(L.shape)
    return (x.unsqueeze(-1) * L_inv).sum(-2).pow(2.0).sum(-1)

In [7]:
VI = torch.inverse(cov(X)) #inverse of covariance matrix
for _input in X:
    print(_batch_mahalanobis(VI,_input))

tensor(6.6430e-09, grad_fn=<SumBackward1>)
tensor(7.5918e-08, grad_fn=<SumBackward1>)
tensor(2.0314e-08, grad_fn=<SumBackward1>)
tensor(1.2633e-07, grad_fn=<SumBackward1>)
tensor(1.0388e-07, grad_fn=<SumBackward1>)
tensor(2.4629e-08, grad_fn=<SumBackward1>)
tensor(1.4290e-08, grad_fn=<SumBackward1>)
tensor(2.3810e-08, grad_fn=<SumBackward1>)
tensor(4.5422e-08, grad_fn=<SumBackward1>)
tensor(7.3179e-08, grad_fn=<SumBackward1>)
tensor(1.7271e-08, grad_fn=<SumBackward1>)
tensor(9.4369e-09, grad_fn=<SumBackward1>)
tensor(9.7419e-08, grad_fn=<SumBackward1>)
tensor(5.8754e-08, grad_fn=<SumBackward1>)
tensor(1.3643e-07, grad_fn=<SumBackward1>)
tensor(1.9711e-07, grad_fn=<SumBackward1>)
tensor(1.8443e-08, grad_fn=<SumBackward1>)
tensor(1.4760e-08, grad_fn=<SumBackward1>)
tensor(5.3511e-08, grad_fn=<SumBackward1>)
tensor(2.0477e-08, grad_fn=<SumBackward1>)
tensor(4.9527e-09, grad_fn=<SumBackward1>)
tensor(5.4732e-08, grad_fn=<SumBackward1>)
tensor(5.0620e-08, grad_fn=<SumBackward1>)
tensor(3.19